# Week 1 • Day 1 — Ollama Web Summarizer

This notebook mirrors the original Day 1 exercise but uses a local Ollama model via the OpenAI-compatible API.


## Setup

- Load environment variables
- Configure OpenAI-compatible client for Ollama
- Print selected base URL and model

In [ ]:
# Imports and environment
import os
import logging
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

from utils import fetch_website_contents

load_dotenv(override=True)
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434/v1")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3.2")
LOG_LEVEL = os.getenv("LOG_LEVEL", "INFO").upper()

# Configure logging
logging.basicConfig(
    level=getattr(logging, LOG_LEVEL, logging.INFO),
    format="%(asctime)s %(levelname)s %(name)s - %(message)s",
)
logger = logging.getLogger("w1d1")

# Connect to Ollama via OpenAI-compatible API (no real OpenAI calls)
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")
print(f"Using Ollama at {OLLAMA_BASE_URL} with model {OLLAMA_MODEL}")

## Prompts

- Define the system and user prompts
- Helper: `messages_for(website)` builds the chat message list

In [ ]:
# Prompts
system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

def messages_for(website: str):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website},
    ]

## Summarization helpers

- `summarize(url)`: scrape page text and call Ollama
- `display_summary(url)`: render the summary as Markdown

In [ ]:
# Summarization helpers

def summarize(url: str) -> str:
    logger.info(f"Summarizing URL: {url}")
    try:
        website = fetch_website_contents(url)
    except Exception as e:
        logger.error(f"Failed to fetch website contents from {url}: {e}")
        return f"Error: failed to fetch website contents from {url}. Details: {e}"

    try:
        response = ollama.chat.completions.create(
            model=OLLAMA_MODEL,
            messages=messages_for(website),
        )
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"Ollama chat completion failed: {e}")
        return f"Error: summarization failed. Details: {e}"


def display_summary(url: str) -> None:
    try:
        summary = summarize(url)
        display(Markdown(summary))
    except Exception as e:
        logger.exception(f"Unexpected error displaying summary for {url}: {e}")
        display(Markdown(f"**Unexpected error**: {e}"))

## Run

- Read `WEBSITE_URL` from environment (fallback to `https://edwarddonner.com`)
- Generate and display the summary

In [ ]:
# Try it out
# Tip: If llama3.2 is slow on your machine, try: export OLLAMA_MODEL=llama3.2:1b

target_url = os.getenv("WEBSITE_URL", "https://edwarddonner.com")
display_summary(target_url)

## Test GPT-5 via OpenRouter

Optional: Test GPT-5 using OpenRouter API. Requires `OPENROUTER_API_KEY` and optionally `OPENROUTER_BASE_URL` in your `.env` file.

In [ ]:
load_dotenv()  # Loads your .env file

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),  # Use the name you saved in your .env
)

response = client.chat.completions.create(
    model="openai/gpt-5.2",  # Call GPT-5 specifically
    messages=[
        {"role": "system", "content": "You are a senior AI engineer assisting an Andela student."},
        {"role": "user", "content": "Explain how to implement multi-tenancy in Spring Boot."}
    ],
    max_tokens=2000,  # Limit response to stay within credit limits
    # GPT-5.2 supports high-level reasoning effort
    extra_body={
        "reasoning": {"effort": "medium"} 
    }
)

print(response.choices[0].message.content)

## Conclusion

- Aligned with Day 1 (GPT) structure: `system_prompt`, `user_prompt_prefix`, `messages_for`, `summarize`, and `display_summary` mirror the original flow.
- Replaced GPT calls with Ollama via OpenAI-compatible `/v1/chat/completions` while keeping the message schema identical.
- Added logging and error handling around scraping and LLM calls for robustness.
- Configure behavior via environment: `OLLAMA_BASE_URL`, `OLLAMA_MODEL`, `WEBSITE_URL`, `LOG_LEVEL`.

Next steps:
- Try a different model (e.g., `llama3.2:1b`) if resources are limited.
- Extend prompts for alternative tones or languages.
- Add link crawling (see Day 2 pattern) if you want multi-page summaries.